# Week 7 - Numerical integration

## Interpolatory quadrature

### Lagrange basis polynomials

Given three distinct points $(x_0, y_0)$, $(x_1, y_1)$, $(x_2, y_2)$, how do we find the degree two polynomial which passes through them?

One way to view this: how do we find $a$, $b$, and $c$ such that

$$p(x_i) = a x_i^2 + b x_i + c = y_i$$

for each $i \in \left\{ 0, 1, 2 \right\}$?

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

x = np.array([-1, 0, 1], dtype=float)
y = np.array([1, 2, -2], dtype=float)

c = np.polyfit(x, y, 2)

print(f"{c=}")

x_plot = np.linspace(-1, 1, 1000)
y_plot = np.polyval(c, x_plot)

fig, ax = plt.subplots(1)
ax.plot(x, y, "kx")
ax.plot(x_plot, y_plot, "r-")
ax.axhline(0, color="#888888")
ax.axvline(0, color="#888888")
ax.set_xlim(x_plot[0], x_plot[-1])
ax.set_xlabel("$x$", fontsize="x-large")
ax.set_ylabel("$p(x)$", fontsize="x-large")

An *alternative* way to view this problem. Let's first define three different polynomials

$$l_0(x) = a_0 x^2 + b_0 x + c_0,$$
$$l_1(x) = a_1 x^2 + b_1 x + c_1,$$
$$l_2(x) = a_2 x^2 + b_2 x + c_2,$$

defined such that

$$l_i ( x_j ) = \begin{cases} 1 & \text{if } i = j \\ 0 & \text{otherwise} \end{cases}.$$

These are *Lagrange basis polynomials*. Then we must have

$$p(x) = y_0 l_0(x) + y_1 l_1(x) + y_2 l_2(x).$$

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

x = np.array([-1, 0, 1], dtype=float)
y = np.array([1, 2, -2], dtype=float)


def l_v(n, i):
    l_v = np.zeros(n, dtype=float)
    l_v[i] = 1
    return l_v


c_lagrange = [np.polyfit(x, l_v(3, i), 2)
              for i in range(3)]

x_plot = np.linspace(-1, 1, 1000)

fig, ax = plt.subplots(1, 3, figsize=(12, 4), sharey=True)
for i in range(3):
    ax[i].plot(x, l_v(3, i), "kx")
    ax[i].plot(x_plot, np.polyval(c_lagrange[i], x_plot), "r-")
    ax[i].axhline(0, color="#888888")
    ax[i].axvline(0, color="#888888")
    ax[i].set_xlim(x_plot[0], x_plot[-1])
    ax[i].set_xlabel("$x$", fontsize="x-large")
    ax[i].set_ylabel(f"$l_{{{i}}}(x)$", fontsize="x-large")

y_plot = np.zeros_like(x_plot)
for i in range(3):
    y_plot += np.polyval(c_lagrange[i], x_plot) * y[i]

fig, ax = plt.subplots(1)
ax.plot(x, y, "kx")
ax.plot(x_plot, y_plot, "r-")
ax.axhline(0, color="#888888")
ax.axvline(0, color="#888888")
ax.set_xlim(x_plot[0], x_plot[-1])
ax.set_xlabel("$x$", fontsize="x-large")
ax.set_ylabel("$p(x)$", fontsize="x-large")

### Interpolatory quadrature rules

A general $N$-point *quadrature rule* on the interval $x \in [-1, 1]$ takes the form

$$\int_{-1}^1 f(x) dx \approx \sum_{i = 0}^{N - 1} w_i f(x_i),$$

where the $x_i$ are the *quadrature points* and the $w_i$ are the *quadrature weights*.

Considering the maximal degree $2$ case, we define an interpolating polynomial $p(x)$ such that

\begin{align*}
    p(x_0) & = f(x_0) = y_0, \\
    p(x_1) & = f(x_1) = y_1, \\
    p(x_2) & = f(x_2) = y_2.
\end{align*}

We use this to approximate the integral.

\begin{align*}
  \int_{-1}^1 f(x) dx
    & \approx \int_{-1}^1 p(x) dx \\
    & = \int_{-1}^1 \left[ y_0 l_0(x) + y_1 l_1(x) + y_2 l_2(x) \right] dx \\
    & = y_0 \int_{-1}^1 l_0(x) dx + y_1 \int_{-1}^1 l_1(x) dx + y_2 \int_{-1}^1 l_2(x) dx \\
    & = y_0 w_0 + y_1 w_1 + y_2 w_2 \\
    & = \sum_{i = 1}^2 w_i f(x_i),
\end{align*}

with quadrature weights

\begin{align*}
    w_0 & = \int_{-1}^1 l_0(x) dx, \\
    w_1 & = \int_{-1}^1 l_1(x) dx, \\
    w_2 & = \int_{-1}^1 l_2(x) dx.
\end{align*}

In [ ]:
def w(c_lagrange):
    c_int = np.polyint(c_lagrange)
    return np.polyval(c_int, 1) - np.polyval(c_int, -1)


print(f"{w(c_lagrange[0])=}")
print(f"{w(c_lagrange[1])=}")
print(f"{w(c_lagrange[2])=}")

Question: When is a quadrature rule of the form

$$\int_{-1}^1 f(x) dx \approx \sum_{i = 0}^{N - 1} w_i f(x_i)$$

*not* an interpolatory quadrature rule?

## Composite quadrature rules

To apply a quadrature rule to a general interval $[a, b]$ we can use a change of variables,

$$x' = \frac{2}{b - a} ( x - a ) - 1.$$

Then

\begin{align*}
  \int_a^b f(x) dx
    & = \int_{x=-1}^{x=1} f ( x ( x' ) ) \frac{dx}{dx'} dx' \\
    & = \frac{b - a}{2} \int_{-1}^1 f \left( \frac{b - a}{2} ( x' + 1 ) + a \right) dx'.
\end{align*}

which suggests

$$\int_{a}^b f(x) dx \approx \frac{b - a}{2} \sum_{i = 0}^{N - 1} w_i f \left( \frac{b - a}{2} ( x' + 1 ) + a \right).$$

We can now use this to define a *composite* quadrature rule, applying some quadrature rule within different sub-intervals.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np


def f(x):
    return np.exp(-x) * np.cos(x) ** 2


a = 0
b = np.pi
integral_exact = 0.6 * (1 - np.exp(-np.pi))

In [ ]:
N_intervals = 3
x_plot = np.linspace(a, b, 1000)
x_quad = np.linspace(a, b, N_intervals + 1)

fig, ax = plt.subplots(1)
ax.plot(x_plot, f(x_plot), "k-")
ax.axvline(x_quad[0], color="r", linewidth=1, linestyle=":")
for i in range(N_intervals):
    ax.axvline(x_quad[i + 1], color="r", linewidth=1, linestyle=":")
    x_quad_i = np.array([x_quad[i],
                        0.5 * (x_quad[i] + x_quad[i + 1]),
                        x_quad[i + 1]],
                        dtype=float)
    x_plot_i = np.linspace(x_quad[i], x_quad[i + 1], 100)
    plt.plot(x_quad_i, f(x_quad_i), "kx", markersize=4)
    plt.plot(x_plot_i,
             np.polyval(np.polyfit(x_quad_i, f(x_quad_i), 2), x_plot_i),
             "r-")
ax.axhline(0, color="#888888")
ax.axvline(0, color="#888888")
ax.set_xlim(x_plot[0], x_plot[-1])
ax.set_xlabel("$x$", fontsize="x-large")

In [ ]:
for N_intervals in [1, 2, 4, 8, 16, 32, 64]:
    x_quad = np.linspace(a, b, N_intervals + 1)

    # Composite Simpson's rule
    # Note: With Simpson's rule it's common in practice to define a set of
    # points (here the x_quad) which include the mid-point values
    integral_approx = 0
    for i in range(N_intervals):
        integral_approx += (0.5 * (x_quad[i + 1] - x_quad[i])
                            * (f(x_quad[i])
                               + 4 * f(0.5 * (x_quad[i] + x_quad[i + 1]))
                               + f(x_quad[i + 1])) / 3)
    # integral_approx = (0.5 * (b - a) / N_intervals
    #                    * (((f(x_quad[0]) + f(x_quad[-1]))
    #                        + 2 * sum(f(x_quad[1:-1]))
    #                        + 4 * sum(f(0.5 * (x_quad[1:] + x_quad[:-1]))))
    #                       / 3))

    print(f"{N_intervals=}")
    print(f"    {integral_exact=}")
    print(f"    {integral_approx=}")
    print(f"    {abs(integral_exact - integral_approx)=}")